In [1]:
from pathlib import Path

from repath.utils.paths import project_root

experiment_name = "lee"
experiment_root = project_root() / "experiments" /  experiment_name

In [2]:
import pytorch_lightning as pl
from torchvision.models import densenet121
import torch.nn as nn

class PatchClassifier(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()
        model = densenet121(pretrained=True)
        model.classifier = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1000, bias=True),
            nn.Linear(1000, 2))
        self.model = model

    def step(self, batch, batch_idx, label):
        x, y = batch
        logits = self.model(x)

        criterion = nn.CrossEntropyLoss()
        loss = criterion(logits, y)
        self.log(f"{label}_loss", loss)

        pred = torch.log_softmax(logits, dim=1)
        correct=pred.argmax(dim=1).eq(y).sum().item()
        total=len(y)   
        accu = correct / total
        self.log(f"{label}_accuracy", accu)
        
        return loss

    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "train")

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "val")

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(), 
                                    lr=0.1, 
                                    momentum=0.9, 
                                    weight_decay=0.0001)           
        scheduler = {
            'scheduler': torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1),
            'interval': 'epoch' 
        }
        return [optimizer], [scheduler]
    
    def forward(self, x):
        return self.model(x)

In [3]:
# calculate false positives
cp_path = list((experiment_root / "patch_model").glob("*.ckpt"))[0]
print(cp_path)
classifier = PatchClassifier.load_from_checkpoint(checkpoint_path=cp_path)

output_dir16 = experiment_root / "train_index16" / "pre_hnm_results"
output_dir17 = experiment_root / "train_index17" / "pre_hnm_results"

results_dir_name = "results"
heatmap_dir_name = "heatmaps"

/home/ubuntu/repath/experiments/lee/patch_model/checkpoint.ckpt-v0.ckpt


In [4]:
from repath.preprocess.patching import SlidesIndex
import repath.data.datasets.camelyon16 as camelyon16
import repath.data.datasets.camelyon17 as camelyon17

train16 = SlidesIndex.load(camelyon16.training(), experiment_root / "train_index16")
train17 = SlidesIndex.load(camelyon17.training(), experiment_root / "train_index17")

In [5]:
from torchvision.transforms import Compose, ToTensor, RandomCrop, Normalize

transform = Compose([
    RandomCrop((240, 240)),
    ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#transform = Compose([
#    RandomCrop((240, 240)),
#    ToTensor()
#])

In [6]:
from repath.data.slides.slide import Region

def to_patch(p: tuple):
        region = Region.patch(p.x, p.y, sps.patch_size, sps.level)
        image = slide.read_region(region)
        image = image.convert('RGB')
        return image
    
#classifier.freeze()
    
#for sps in train16:       
#    with sps.open_slide() as slide:
#        for p in sps:
#            test_patch = to_patch(p)
#            test_patch = transform(test_patch)
#            print(test_patch.size())
#            out = classifier(test_patch)
#            print(out)

In [7]:
from repath.preprocess.patching import SlidesIndexResults

train_results16 = SlidesIndexResults.predict_dataset(train16, classifier, 128, 8, transform, output_dir16, results_dir_name, heatmap_dir_name)
train_results16.save_results_index()

Let's use 8 GPUs!
Batch 0 of 393
Batch 100 of 393
Batch 200 of 393
Batch 300 of 393
Saving normal/normal_050.tif
(862, 381)
/home/ubuntu/repath/experiments/lee/train_index16/pre_hnm_results/heatmaps/normal/normal_050.png


KeyboardInterrupt: 

In [ ]:
train_results17 = SlidesIndexResults.predict_dataset(train17, classifier, 128, 8, output_dir17, results_dir_name, heatmap_dir_name)
train_results17.save_results_index()

In [ ]:
    

    train_results = train_results.as_combined()

    patch_dict = train_results.dataset.labels
    FP_mask = np.logical_and(train_results.patches_df['tumor'] > 0.5,
                             train_results.patches_df['label'] == patch_dict['tumor'])
    train_results.patches_df = train_results.patches_df[FP_mask]
    ### To do put in a limit on how many patches to add
    train_results.save_patches(experiment_root / "training_patches")

    # prepare our data
    batch_size = 128
    train_set = ImageFolder.load(experiment_root / "training_patches")
    valid_set = ImageFolder.load(experiment_root / "validation_patches")
    train_loader = DataLoader(train_set, batch_size=batch_size)
    valid_loader = DataLoader(valid_set, batch_size=batch_size)

    # configure logging and checkpoints
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=experiment_root / "patch_model_hnm",
        filename=f"checkpoint-{epoch:02d}-{val_loss:.2f}.ckpt",
        save_top_k=1,
        mode="min",
    )

    # train our model
    cp_path = (experiment_root / "patch_model").glob("*.ckpt")[0]
    model = Backbone().load_from_checkpoint(cp_path)
    classifier = PatchClassifier(model)
    trainer = pl.Trainer(callbacks=[checkpoint_callback])
    trainer.fit(classifier, train_dataloader=train_loader, val_dataloaders=valid_loader)